# Word2Vec (Skipgram )

In [113]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd

In [114]:
np.__version__, torch.__version__

('1.26.4', '2.5.1+cu121')

In [115]:
import matplotlib
matplotlib.__version__

'3.10.0'

In [116]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
#import os
import os

os.chdir('/content/drive/MyDrive/_NLP/NLP-A1-That-s-What-I-LIKE-st125553')

## 1. Load data

In [118]:
# Load nltk
import nltk

# download news category dataset from nltk
nltk.download('brown') # download brown corpus
nltk.download('punkt') # download punkt for tokenization
nltk.download('punkt_tab') # download punkt_tab for tokenization

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [119]:
#1. tokenization
# import the news category dataset
from nltk.corpus import brown
corpus = brown.sents(categories='news')
corpus

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [120]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

print(f"before vocabs_len: {len(vocabs)}")
vocabs.append('<UNK>')
print(f"after vocabs_len: {len(vocabs)}")

before vocabs_len: 14394
after vocabs_len: 14395


In [121]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}
#word2index['<UNK>'] # already in the system

In [122]:
index2word = {v:k for k, v in word2index.items()}
index2word[14394]

'<UNK>'

## 2. Prepare train data

In [123]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus, window_size=2):

    skipgrams = []
    outside = []
    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(window_size, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]

            #outside words based on window size
            outside = [word2index[doc[i+j]] for j in range(-window_size, window_size+1) if i+j != i]

            #for each of these two outside words, we gonna append to a list
            for each_out in outside:
                skipgrams.append([center, each_out])
                #center, outside1;   center, outside2

    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)

    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])

    skipgrams

    return np.array(inputs), np.array(labels), skipgrams

In [124]:
window_size = 2
batch_size =   2
x, y , skipgrams = random_batch(batch_size, corpus, window_size)
#skipgrams

In [125]:
x.shape  #batch_size, 1

(2, 1)

In [126]:
y.shape  #batch_size 1

(2, 1)

In [127]:
x, y

(array([[4449],
        [9558]]),
 array([[4480],
        [8234]]))

## 3. Model

$$J(\theta) = -\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m \leq j \leq m \\ j \neq 0}}\log P(w_{t+j} | w_t; \theta)$$

where $P(w_{t+j} | w_t; \theta) = $

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

where $o$ is the outside words and $c$ is the center word

In [128]:
voc_size = len(vocabs)
voc_size

14395

In [129]:
embedding = nn.Embedding(voc_size, 2)

In [130]:
x_tensor = torch.LongTensor(x)
embedding(x_tensor).shape  #(batch_size, 1, emb_size)

torch.Size([2, 1, 2])

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

In [131]:
class Skipgram(nn.Module):

    def __init__(self, voc_size, emb_size, word2index):
        super(Skipgram, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.word2index = word2index

    def forward(self, center, outside, all_vocabs):
        center_embedding     = self.embedding_center(center)  #(batch_size, 1, emb_size)
        outside_embedding    = self.embedding_center(outside) #(batch_size, 1, emb_size)
        all_vocabs_embedding = self.embedding_center(all_vocabs) #(batch_size, voc_size, emb_size)

        top_term = torch.exp(outside_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2))
        #batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1)

        lower_term = all_vocabs_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2)
        #batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, vocab_size)

        lower_term_sum = torch.sum(torch.exp(lower_term), 1)  #(batch_size, 1)

        loss = -torch.mean(torch.log(top_term / lower_term_sum))  #scalar

        return loss

    # def get_embed(self, word):
    #     try:
    #         index = self.word2index[word]
    #     except:
    #         index = self.word2index['<UNK>']

    #     word = torch.LongTensor([self.word2index[word]])

    #     embed_c = self.embedding_center(word)
    #     embed_o = self.embedding_outside(word)
    #     embed   = (embed_c + embed_o) / 2
    #     return embed[0][0].item(), embed[0][1].item()

    def get_embed(self, word):
        try:
            index = self.word2index[word]
        except KeyError: # More specific exception handling
            index = self.word2index['<UNK>'] # If not found, default to <UNK>

        # Use the index obtained, not looking up the word again
        word_tensor = torch.LongTensor([index])

        embed_c = self.embedding_center(word_tensor)
        embed_o = self.embedding_outside(word_tensor)
        embed   = (embed_c + embed_o) / 2
        return embed[0][0].item(), embed[0][1].item()


In [132]:
#prepare all vocabs

batch_size = 2
print(voc_size)

def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>'], seq))
    return torch.LongTensor(idxs)

# Ensure the vocab size matches
all_vocabs = prepare_sequence(list(vocabs[:voc_size]), word2index).expand(batch_size, voc_size)
all_vocabs

14395


tensor([[    0,     1,     2,  ..., 14392, 14393, 14394],
        [    0,     1,     2,  ..., 14392, 14393, 14394]])

In [133]:
model = Skipgram(voc_size, 2, word2index)
model

Skipgram(
  (embedding_center): Embedding(14395, 2)
  (embedding_outside): Embedding(14395, 2)
)

In [134]:
input_tensor = torch.LongTensor(x)
label_tensor = torch.LongTensor(y)
#label_tensor

In [135]:
loss = model(input_tensor, label_tensor, all_vocabs)

In [136]:
loss

tensor(10.9722, grad_fn=<NegBackward0>)

## 4. Training

In [137]:
batch_size = 2
emb_size   = 2
window_size = 2
model      = Skipgram(voc_size, emb_size, word2index)
optimizer  = optim.Adam(model.parameters(), lr=0.001)

In [138]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [139]:
import time

num_epochs = 100
total_loss = 0
start_time = time.time()
for epoch in range(num_epochs):
    start = time.time()

    #get batch
    input_batch, label_batch, skipgrams = random_batch(batch_size, corpus, window_size)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)

    #predict
    loss = model(input_tensor, label_tensor, all_vocabs)

    #backprogate
    optimizer.zero_grad()
    loss.backward()

    #update alpha
    optimizer.step()

    end = time.time()

    #print time
    epoch_mins, epoch_secs = epoch_time(start, end)

    #print the loss
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1} | Loss: {loss} | time: {epoch_mins}m {epoch_secs}s")

    #calculate loss
    #total_loss += loss.item()

    #calculate avg loss
    #avg_loss = total_loss / (epoch + 1)

# print loss
print(f"Final Loss: {loss:2.6f}")

# print(f"Total loss: {total_loss:2.6f}")
# print(f"Avg loss: {avg_loss:2.6f}")
# calculate time
end_time = time.time()
minutes, seconds = epoch_time(start_time, end_time)
print(f"Total time: {minutes:2.0f} minutes {seconds:2.0f} seconds")


Epoch: 10 | Loss: 9.97850513458252 | time: 0m 1s
Epoch: 20 | Loss: 12.385770797729492 | time: 0m 0s
Epoch: 30 | Loss: 10.53164291381836 | time: 0m 1s
Epoch: 40 | Loss: 10.718771934509277 | time: 0m 0s
Epoch: 50 | Loss: 9.652607917785645 | time: 0m 0s
Epoch: 60 | Loss: 10.15438461303711 | time: 0m 1s
Epoch: 70 | Loss: 9.85429573059082 | time: 0m 1s
Epoch: 80 | Loss: 10.21426010131836 | time: 0m 0s
Epoch: 90 | Loss: 10.8720064163208 | time: 0m 1s
Epoch: 100 | Loss: 10.671304702758789 | time: 0m 0s
Final Loss: 10.671305
Total time:  1 minutes 30 seconds


## 5. Testing


In [140]:
vect = []

for word in vocabs:
    vect.append(model.get_embed(word))
vect = np.array(vect)

In [141]:
#scipy version
from scipy import spatial

def cos_sim(a, b):
    cos_sim = 1 - spatial.distance.cosine(a, b)  #distance = 1 - similarlity, because scipy only gives distance
    return cos_sim

def cos_sim_scores(vect_space, target_vect):
    scores = []
    for each_vect in vect_space:
        each_vect = tuple(each_vect)
        target_vect=tuple(target_vect)
        scores.append(cos_sim(target_vect, each_vect))

    return np.array(scores)

In [95]:
def similarity(model, data):
    words = data.split(" ")

    embed0 = np.array(model.get_embed(words[0]))
    embed1 = np.array(model.get_embed(words[1]))
    embed2 = np.array(model.get_embed(words[2]))

    sim_vect = embed1 - embed0 + embed2

    sim_scores = cos_sim_scores(vect, sim_vect)
    max_score_idx = np.argmax(sim_scores)
    sim_word = index2word[max_score_idx]

    result = False
    if sim_word == words[3]:
        result = True

    return result

### Semantic Test

In [96]:
semantic_file = "data/word-test-semantic.txt"
# open file
with open(semantic_file, "r") as file:
    sem_file = file.readlines()
    #send semantic into vector

semantic = []
for sent in sem_file:
    semantic.append(sent.strip())

#semantic

In [97]:
sem_count = len(semantic)
#sem_total
sem_correct = 0
for sent in semantic:
    if similarity(model, sent):
        sem_correct += 1

In [98]:
sem_accuracy = sem_correct / sem_count
print(f"Semantic accuracy: {sem_accuracy:2.2f}")
print(f"Semantic correct: {sem_correct}")
print(f"Semantic count: {sem_count}")

Semantic accuracy: 0.00
Semantic correct: 0
Semantic count: 506


### Syntatic Test

In [99]:
syntatic_file = "data/word-test-syntatic.txt"
# open file
with open(syntatic_file, "r") as file:
    syn_file = file.readlines()

syntatic = []
for sent in syn_file:
    syntatic.append(sent.strip())
#syntatic

In [100]:
syn_count = len(syntatic)
syn_correct = 0
for sent in syntatic:
    if similarity(model, sent):
        syn_correct += 1

In [101]:
syn_accuracy = syn_correct / syn_count
print(f"Syntatic accuracy: {syn_accuracy:2.2f}")
print(f"Syntatic correct: {syn_correct}")
print(f"Syntatic count: {syn_count}")

Syntatic accuracy: 0.00
Syntatic correct: 0
Syntatic count: 1560


### Similarity Test


In [102]:
similarity_file = "data/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"
# open file
with open(similarity_file, "r") as file:
    sim_file = file.readlines()

similarity = []
for sent in sim_file:
    similarity.append(sent.strip())
#syntatic

In [103]:
def similarity_test(model, test_data):
    words = test_data.split("\t")

    embed0 = np.array(model.get_embed(words[0].strip()))
    embed1 = np.array(model.get_embed(words[1].strip()))

    model_result = embed1 @ embed0.T
    sim_result = float(words[2].strip())

    return sim_result, model_result

In [104]:
sim_scores = []
model_scores = []
for sent in similarity:
    sim_result, model_result = similarity_test(model, sent)

    sim_scores.append(sim_result)
    model_scores.append(model_result)

In [105]:
from scipy.stats import spearmanr

corr = spearmanr(sim_scores, model_scores)[0]

print(f"The correlation result is {corr:2.2f}.")

The correlation result is 0.19.


## 6. Save the model


In [106]:
# Saving the model for testing
torch.save(model.state_dict(), 'app/models/w2v-skipgram.model')

In [142]:
skipgram_args = {
    'voc_size': voc_size,
    'emb_size': emb_size,
    'word2index': word2index,
}

In [143]:
import pickle
pickle.dump(skipgram_args, open('app/models/w2v-skipgram.args', 'wb'))

In [144]:
skg_args = pickle.load(open('app/models/w2v-skipgram.args', 'rb'))
load_model = Skipgram(**skipgram_args)
load_model.load_state_dict(torch.load('app/models/w2v-skipgram.model'))

<ipython-input-144-8165938d23cf>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model.load_state_dict(torch.load('app/models/w2v-skipgram.model'))


<All keys matched successfully>

In [145]:
keyword = load_model.get_embed('The')
keyword

(-0.8691345453262329, 0.7977749109268188)

In [ ]:
# select similar wrod from model result
